In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.layers import Dropout,BatchNormalization,LSTM,Bidirectional,GlobalMaxPool1D,Input,Activation,Flatten,Embedding,Dense,concatenate,Conv1D,MaxPooling1D
import string
import re
from tqdm import tqdm
import nltk
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
import spacy
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,f1_score,recall_score,precision_score,classification_report
import os
from keras.models import Model
import seaborn as sns
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import warnings
warnings.filterwarnings('ignore')
import tensorboard
from textblob import TextBlob
import os
import tensorboard
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import datetime
from keras.initializers import he_normal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from prettytable import PrettyTable











In [3]:
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 29.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 48.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.7/263.7 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 68.1 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 37.1 MB/s eta 0:00:0000:01:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ktrain: filename=ktrain-0.31.6-py3-none-any.whl size=25312758

In [4]:
data_new=pd.read_csv("../input/data-pre-3/data_preprocessed.csv")
data_new.shape

(1010660, 10)

In [5]:
data_new1=data_new.drop(['Unnamed: 0'],axis=1)
data_new1=data_new1.dropna()

In [ ]:
data=data_new1.sample(100000)

In [6]:

y=data_new1['label']
X=data_new1.drop(['label'],axis=1)

In [7]:
X_train,X_test,y_train,y_test=train_test_split(X,y,stratify=y,random_state=42,test_size=0.3,shuffle=True)
print((X_train.shape),y_train.shape)
print((X_test.shape),y_test.shape)

(705968, 8) (705968,)
(302559, 8) (302559,)


In [8]:
class_names=[0,1]

y_train=np.asarray(y_train)
y_test=np.asarray(y_test)

In [9]:
import ktrain
from ktrain import text
model_name = 'distilbert-base-uncased'
t = text.Transformer(model_name, maxlen=40, class_names=class_names)
trn = t.preprocess_train(list(X_train['comment']), y_train)
val = t.preprocess_test(list(X_test['comment']), y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=512)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

preprocessing train...
language: en
train sequence lengths:
	mean : 6
	95percentile : 14
	99percentile : 21


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 6
	95percentile : 14
	99percentile : 21


Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

In [10]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/4
1379/1379 [==============================] - 1949s 1s/step - loss: 0.6000 - accuracy: 0.6688 - val_loss: 0.5605 - val_accuracy: 0.7065
Epoch 2/4
1379/1379 [==============================] - 1937s 1s/step - loss: 0.5503 - accuracy: 0.7149 - val_loss: 0.5407 - val_accuracy: 0.7234
Epoch 3/4
1379/1379 [==============================] - 1934s 1s/step - loss: 0.5226 - accuracy: 0.7368 - val_loss: 0.5337 - val_accuracy: 0.7295
Epoch 4/4
1379/1379 [==============================] - 1940s 1s/step - loss: 0.4948 - accuracy: 0.7567 - val_loss: 0.5378 - val_accuracy: 0.7294


In [ ]:
# learner.model.save_pretrained('.// model_bert')

In [11]:
learner.validate(class_names=t.get_classes())

              precision    recall  f1-score   support

           0       0.72      0.74      0.73    151122
           1       0.74      0.72      0.73    151437

    accuracy                           0.73    302559
   macro avg       0.73      0.73      0.73    302559
weighted avg       0.73      0.73      0.73    302559



array([[112177,  38945],
       [ 42927, 108510]])

In [12]:
predictor1 = ktrain.get_predictor(learner.model,preproc=t).save('model123')

In [13]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [14]:
predictor.predict('ayush is awesome')

0

In [15]:
predictor.predict('I work 40 hours a week for me to be this poor.')

0

In [16]:
predictor.predict('ayush khandelwal')

0

In [17]:
predictor.predict('mohit is awesome')

0

In [18]:
predictor.save('.//model_bert_1')

In [19]:
reloaded_predictor = ktrain.load_predictor('.//model_bert_1')

In [20]:
reloaded_predictor.predict('mohit is awesome')

0

In [21]:
reloaded_predictor.predict('Thats just what I needed today!')

1

In [22]:
reloaded_predictor.predict('Well, what a surprise.')

1

In [23]:
reloaded_predictor.predict('Is it time for your medication or mine?')

0

In [24]:
reloaded_predictor.predict('Really, Sherlock? No! You are clever.')

1

In [25]:
reloaded_predictor.predict('mohit is awesome')

0

In [26]:
reloaded_predictor.predict('Nice perfume. How long did you marinate in it?')

0

In [27]:
reloaded_predictor.predict('Very good; well done!')

0

In [28]:
reloaded_predictor.predict('I like spaghetti.')

0

In [29]:
reloaded_predictor.predict('He reads many books.')

0

In [30]:
reloaded_predictor.predict('I like spaghetti for dinner.')

0

In [ ]:
predictor1 = ktrain.get_predictor(learner.model,preproc=t).save('model123')


# predictor1 = ktrain.load_predictor('model')
# x = "Hello World"
# prediction = predictor.model.predict(x)
# print(prediction)

In [ ]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=train_b.target_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

In [ ]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=128)

In [ ]:
model.summary()

In [ ]:
hist = learner.fit_onecycle(1e-5, 1)